In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import networkx as nx
from matplotlib.pyplot import figure
from pyvis.network import Network
from community import community_louvain


In [ ]:
#uploading the dataframes
filtred = pd.read_pickle('filtred.pkl') #read_csv('filtred.csv', dtype={'Cited names': list})
sent = pd.read_csv('sent.csv')
received = pd.read_csv('received.csv')

In [ ]:
#Last minute manual cleaning
filtred = filtred.loc[filtred['Sender'].str.contains('Gal. 168')==False]
filtred = filtred.loc[filtred['Sender'].str.contains('Epigra')==False]

#Removing 134 letters sent to unidentified receivers
filtred = filtred.loc[filtred['Receiver'].str.contains('on iden')==False]


filtred['Receiver'] = filtred['Receiver'].str.replace(r'Molara Bruto \(della\)', 'Molara Bruto della', regex=True)
filtred['Sender'] = filtred['Sender'].str.replace(r'Molara Bruto \(della\)', 'Molara Bruto della', regex=True)

filtred['Receiver'] = filtred['Receiver'].str.replace(r'Barbarigo Gregorio Giovanni Gasparo', 'Gregorio Barbarigo santo', regex=True)
filtred['Sender'] = filtred['Sender'].str.replace(r'Barbarigo Gregorio Giovanni Gasparo', 'Gregorio Barbarigo santo', regex=True)

filtred['Receiver'] = filtred['Receiver'].str.replace(r"\(de'\)", "de'", regex=True)
filtred['Sender'] = filtred['Sender'].str.replace(r"\(de'\)", "de'", regex=True)


for index, row in filtred.iterrows():
    if isinstance(row['Cited names'], __builtins__.list) == True:
        for i in range(len(row['Cited names'])):
            if row['Cited names'][i]=='Torricelli':
                row['Cited names'][i]='Torricelli Evangelista'
            if 'not Melchi' in row['Cited names'][i]:
                row['Cited names'][i]='Thevenot Melchisédec'
            if 'iani Franc' in row['Cited names'][i]:
                row['Cited names'][i]='Viviani Francesco'
            if row['Cited names'][i]=='Dati':
                row['Cited names'][i]='Dati Carlo Roberto'

In [ ]:
#Remove when year not available
filtred_by_year = filtred.loc[filtred['Year'].str.contains('sd')==False]
filtred_by_year = filtred_by_year.replace(r'^\s*$', np.nan, regex=True)
filtred_by_year = filtred_by_year.dropna(subset=['Year'])

# For Viviani's whole life

In [ ]:
#list of all correspondents
correspondents = filtred_by_year['Sender'].append(filtred_by_year['Receiver']).unique()
#correspondents

#corresponding labels
labels_corres = {}
for i in range(0, len(correspondents)):
    labels_corres.update({correspondents[i]: correspondents[i]})

#list of the top 20 direct correspondents of Viviani   
top_correspondents = filtred_by_year['Sender'].append(filtred_by_year['Receiver']).value_counts().nlargest(20).index.tolist()

#corresponding labels
labels_top_corres = {}
for i in range(0, len(top_correspondents)):
    labels_top_corres.update({top_correspondents[i]: top_correspondents[i]})

#correspondents without top n
corres_wo_top =[]
for i in range(len(correspondents)):
    if correspondents[i] not in top_correspondents:
        corres_wo_top.append(correspondents[i])
        
#corresponding labels
labels_wo_top = {}
for i in range(0, len(corres_wo_top)):
    labels_wo_top.update({corres_wo_top[i]: corres_wo_top[i]})

#list of names only cited
only_cited = []
for index, row in filtred_by_year.iterrows():
    if isinstance(row['Cited names'], __builtins__.list) == True:
        for i in range(len(row['Cited names'])):
            if row['Cited names'][i] not in correspondents.tolist():
                if row['Cited names'][i] not in only_cited:
                    only_cited.append(row['Cited names'][i])

#corresponding labels
labels_only_cited = {}
for i in range(0, len(only_cited)):
    labels_only_cited.update({only_cited[i]: only_cited[i]})

In [ ]:
#creating the graph and adding the starting nodes
G = nx.Graph()

weight_direct = 5
weight_citation_made = 7
weight_citation_received = 1

#going through each row and adding a node/edge from receiver and names cited
for index, row in filtred_by_year.iterrows():

    if G.has_node(row['Sender'])==False:
        G.add_node(row['Sender'])
    
    if G.has_node(row['Receiver']):
        if G.has_edge(row['Sender'], row['Receiver']):
            #increase weight if receiver already a node and edged with sender
            G[row['Sender']][row['Receiver']]['weight'] += weight_direct
        else:
            #create edge if inexistant
            G.add_edge(row['Sender'], row['Receiver'], weight = weight_direct, color = 'r')
    else:
        #create node for receiver if inexistant, and edge to sender
        G.add_node(row['Receiver'])
        G.add_edge(row['Sender'], row['Receiver'], weight = weight_direct, color = 'r')
    
    if isinstance(row['Cited names'], __builtins__.list) == True:
        
#        if row['Sender']=='Viviani Vincenzo':
#            for node in range(0, len(row['Cited names'])):
#                if G.has_edge(row['Receiver'], row['Cited names'][node]):
#                    G[row['Receiver']][row['Cited names'][node]]['weight'] += weight_citation_received
#                elif G.has_node(row['Cited names'][node]):
#                    G.add_edge(row['Receiver'], row['Cited names'][node], weight = weight_citation_received, color = 'b')
#                else:
#                    G.add_node(row['Cited names'][node])
#                    G.add_edge(row['Receiver'], row['Cited names'][node], weight = weight_citation_received, color='b')
#        else:
#            for node in range(0, len(row['Cited names'])):
#                if G.has_edge(row['Sender'], row['Cited names'][node]):
#                    G[row['Sender']][row['Cited names'][node]]['weight'] += weight_citation_made
#                elif G.has_node(row['Cited names'][node]):
#                    G.add_edge(row['Sender'], row['Cited names'][node], weight = weight_citation_made, color = 'b')
#                else:
#                    G.add_node(row['Cited names'][node])
#                    G.add_edge(row['Sender'], row['Cited names'][node], weight = weight_citation_made, color='b')


            
        for node in range(0, len(row['Cited names'])):
            if G.has_edge(row['Sender'], row['Cited names'][node]):
                G[row['Sender']][row['Cited names'][node]]['weight'] += weight_citation_made
            else:
                if G.has_node(row['Cited names'][node]):
                    G.add_edge(row['Sender'], row['Cited names'][node], weight = weight_citation_made, color = 'b')
                else:
                    G.add_node(row['Cited names'][node])
                    G.add_edge(row['Sender'], row['Cited names'][node], weight = weight_citation_made, color='b')
            
            if G.has_edge(row['Receiver'], row['Cited names'][node]):
                G[row['Receiver']][row['Cited names'][node]]['weight'] += weight_citation_received
            else:
                if G.has_node(row['Cited names'][node]):
                    G.add_edge(row['Receiver'], row['Cited names'][node], weight = weight_citation_received, color = 'b')
                else:
                    G.add_node(row['Cited names'][node])
                    G.add_edge(row['Receiver'], row['Cited names'][node], weight = weight_citation_received, color='b')


                    

edges_corres = [x for x in G.edges if (x[0] in correspondents and x[1] in correspondents)]
edges_top_corres = [x for x in G.edges if (x[0] in top_correspondents and x[1] in top_correspondents)]

pos=nx.spring_layout(G, k=.05,iterations=100)#, seed = 7)

figure(figsize=(100,100))#, dpi=400)
weights = [.1*G[u][v]['weight'] for u,v in G.edges]

nx.draw_networkx_nodes(G, pos, node_size=170)#, nodelist=correspondents)
nx.draw_networkx_labels(G, pos, font_size=17, font_family="sans-serif", labels=labels_only_cited)
nx.draw_networkx_labels(G, pos, font_size=17, font_family="sans-serif", font_color='tab:blue', labels=labels_wo_top)
nx.draw_networkx_edges(G, pos, width=weights, alpha=.3, edge_color='tab:blue')#,edgelist=edges_corres)

nx.draw_networkx_labels(G, pos, labels=labels_top_corres, font_size=30, font_family="serif", font_weight='bold', font_color='tab:red')


plt.show()
#plt.savefig("Graph.svg")

# For the Cimento period (1656-1664)

In [ ]:
#Select only letters from period
filtred_by_cimento = filtred_by_year.loc[filtred_by_year['Year'].astype(int) >= 1656]
filtred_by_cimento = filtred_by_cimento.loc[filtred_by_cimento['Year'].astype(int) <= 1663]

#list of all correspondents
correspondents = filtred_by_cimento['Sender'].append(filtred_by_cimento['Receiver']).unique()
#correspondents

#corresponding labels
labels_corres = {}
for i in range(0, len(correspondents)):
    labels_corres.update({correspondents[i]: correspondents[i]})

#list of the top 20 direct correspondents of Viviani   
top_correspondents = filtred_by_cimento['Sender'].append(filtred_by_cimento['Receiver']).value_counts().nlargest(20).index.tolist()

#corresponding labels
labels_top_corres = {}
for i in range(0, len(top_correspondents)):
    labels_top_corres.update({top_correspondents[i]: top_correspondents[i]})

#correspondents without top n
corres_wo_top =[]
for i in range(len(correspondents)):
    if correspondents[i] not in top_correspondents:
        corres_wo_top.append(correspondents[i])
        
#corresponding labels
labels_wo_top = {}
for i in range(0, len(corres_wo_top)):
    labels_wo_top.update({corres_wo_top[i]: corres_wo_top[i]})

#list of names only cited
only_cited = []
for index, row in filtred_by_cimento.iterrows():
    if isinstance(row['Cited names'], __builtins__.list) == True:
        for i in range(len(row['Cited names'])):
            if row['Cited names'][i] not in correspondents.tolist():
                if row['Cited names'][i] not in only_cited:
                    only_cited.append(row['Cited names'][i])

#corresponding labels
labels_only_cited = {}
for i in range(0, len(only_cited)):
    labels_only_cited.update({only_cited[i]: only_cited[i]})

In [ ]:
#creating the graph and adding the starting nodes
G = nx.Graph()

weight_direct = 5
weight_citation_made = 7
weight_citation_received = 1

#going through each row and adding a node/edge from receiver and names cited
for index, row in filtred_by_cimento.iterrows():

    if G.has_node(row['Sender'])==False:
        G.add_node(row['Sender'])
    
    if G.has_node(row['Receiver']):
        if G.has_edge(row['Sender'], row['Receiver']):
            #increase weight if receiver already a node and edged with sender
            G[row['Sender']][row['Receiver']]['weight'] += weight_direct
        else:
            #create edge if inexistant
            G.add_edge(row['Sender'], row['Receiver'], weight = weight_direct, color = 'r')
    else:
        #create node for receiver if inexistant, and edge to sender
        G.add_node(row['Receiver'])
        G.add_edge(row['Sender'], row['Receiver'], weight = weight_direct, color = 'r')
    
    if isinstance(row['Cited names'], __builtins__.list) == True:
        
#        if row['Sender']=='Viviani Vincenzo':
#            for node in range(0, len(row['Cited names'])):
#                if G.has_edge(row['Receiver'], row['Cited names'][node]):
#                    G[row['Receiver']][row['Cited names'][node]]['weight'] += weight_citation_received
#                elif G.has_node(row['Cited names'][node]):
#                    G.add_edge(row['Receiver'], row['Cited names'][node], weight = weight_citation_received, color = 'b')
#                else:
#                    G.add_node(row['Cited names'][node])
#                    G.add_edge(row['Receiver'], row['Cited names'][node], weight = weight_citation_received, color='b')
#        else:
#            for node in range(0, len(row['Cited names'])):
#                if G.has_edge(row['Sender'], row['Cited names'][node]):
#                    G[row['Sender']][row['Cited names'][node]]['weight'] += weight_citation_made
#                elif G.has_node(row['Cited names'][node]):
#                    G.add_edge(row['Sender'], row['Cited names'][node], weight = weight_citation_made, color = 'b')
#                else:
#                    G.add_node(row['Cited names'][node])
#                    G.add_edge(row['Sender'], row['Cited names'][node], weight = weight_citation_made, color='b')


            
        for node in range(0, len(row['Cited names'])):
            if G.has_edge(row['Sender'], row['Cited names'][node]):
                G[row['Sender']][row['Cited names'][node]]['weight'] += weight_citation_made
            else:
                if G.has_node(row['Cited names'][node]):
                    G.add_edge(row['Sender'], row['Cited names'][node], weight = weight_citation_made, color = 'b')
                else:
                    G.add_node(row['Cited names'][node])
                    G.add_edge(row['Sender'], row['Cited names'][node], weight = weight_citation_made, color='b')
            
            if G.has_edge(row['Receiver'], row['Cited names'][node]):
                G[row['Receiver']][row['Cited names'][node]]['weight'] += weight_citation_received
            else:
                if G.has_node(row['Cited names'][node]):
                    G.add_edge(row['Receiver'], row['Cited names'][node], weight = weight_citation_received, color = 'b')
                else:
                    G.add_node(row['Cited names'][node])
                    G.add_edge(row['Receiver'], row['Cited names'][node], weight = weight_citation_received, color='b')

             


edges_corres = [x for x in G.edges if (x[0] in correspondents and x[1] in correspondents)]
edges_top_corres = [x for x in G.edges if (x[0] in top_correspondents and x[1] in top_correspondents)]

pos=nx.spring_layout(G, k=.05,iterations=100)#, seed = 7)

figure(figsize=(100,100))#, dpi=400)
weights = [.1*G[u][v]['weight'] for u,v in G.edges]

nx.draw_networkx_nodes(G, pos, node_size=170)#, nodelist=correspondents)
nx.draw_networkx_labels(G, pos, font_size=17, font_family="sans-serif", labels=labels_only_cited)
nx.draw_networkx_labels(G, pos, font_size=17, font_family="sans-serif", font_color='tab:blue', labels=labels_wo_top)
nx.draw_networkx_edges(G, pos, width=weights, alpha=.5, edge_color='tab:blue')#,edgelist=edges_corres)

nx.draw_networkx_labels(G, pos, labels=labels_top_corres, font_size=30, font_family="serif", font_weight='bold', font_color='tab:red')


plt.show()
#plt.savefig("Graph.svg")

# For the Paris period (1664-1671)

In [ ]:
#Select only letters from period
filtred_by_paris = filtred_by_year.loc[filtred_by_year['Year'].astype(int) >= 1664]
filtred_by_paris = filtred_by_paris.loc[filtred_by_paris['Year'].astype(int) <= 1671]

#list of all correspondents
correspondents = filtred_by_paris['Sender'].append(filtred_by_paris['Receiver']).unique()
#correspondents

#corresponding labels
labels_corres = {}
for i in range(0, len(correspondents)):
    labels_corres.update({correspondents[i]: correspondents[i]})

#list of the top 20 direct correspondents of Viviani   
top_correspondents = filtred_by_paris['Sender'].append(filtred_by_paris['Receiver']).value_counts().nlargest(20).index.tolist()

#corresponding labels
labels_top_corres = {}
for i in range(0, len(top_correspondents)):
    labels_top_corres.update({top_correspondents[i]: top_correspondents[i]})

#correspondents without top n
corres_wo_top =[]
for i in range(len(correspondents)):
    if correspondents[i] not in top_correspondents:
        corres_wo_top.append(correspondents[i])
        
#corresponding labels
labels_wo_top = {}
for i in range(0, len(corres_wo_top)):
    labels_wo_top.update({corres_wo_top[i]: corres_wo_top[i]})

#list of names only cited
only_cited = []
for index, row in filtred_by_paris.iterrows():
    if isinstance(row['Cited names'], __builtins__.list) == True:
        for i in range(len(row['Cited names'])):
            if row['Cited names'][i] not in correspondents.tolist():
                if row['Cited names'][i] not in only_cited:
                    only_cited.append(row['Cited names'][i])

#corresponding labels
labels_only_cited = {}
for i in range(0, len(only_cited)):
    labels_only_cited.update({only_cited[i]: only_cited[i]})

In [ ]:
#creating the graph and adding the starting nodes
G = nx.Graph()

weight_direct = 5
weight_citation_made = 7
weight_citation_received = 1

#going through each row and adding a node/edge from receiver and names cited
for index, row in filtred_by_paris.iterrows():

    if G.has_node(row['Sender'])==False:
        G.add_node(row['Sender'])
    
    if G.has_node(row['Receiver']):
        if G.has_edge(row['Sender'], row['Receiver']):
            #increase weight if receiver already a node and edged with sender
            G[row['Sender']][row['Receiver']]['weight'] += weight_direct
        else:
            #create edge if inexistant
            G.add_edge(row['Sender'], row['Receiver'], weight = weight_direct, color = 'r')
    else:
        #create node for receiver if inexistant, and edge to sender
        G.add_node(row['Receiver'])
        G.add_edge(row['Sender'], row['Receiver'], weight = weight_direct, color = 'r')
    
    if isinstance(row['Cited names'], __builtins__.list) == True:
        
        #Citations not counted for Viviani but only for his correspondents (radial = nb of letters, axial = citations b/w them)
#        if row['Sender']=='Viviani Vincenzo':
#            for node in range(0, len(row['Cited names'])):
#                if G.has_edge(row['Receiver'], row['Cited names'][node]):
#                    G[row['Receiver']][row['Cited names'][node]]['weight'] += weight_citation_received
#                elif G.has_node(row['Cited names'][node]):
#                    G.add_edge(row['Receiver'], row['Cited names'][node], weight = weight_citation_received, color = 'b')
#                else:
#                    G.add_node(row['Cited names'][node])
#                    G.add_edge(row['Receiver'], row['Cited names'][node], weight = weight_citation_received, color='b')
#        else:
#            for node in range(0, len(row['Cited names'])):
#                if G.has_edge(row['Sender'], row['Cited names'][node]):
#                    G[row['Sender']][row['Cited names'][node]]['weight'] += weight_citation_made
#                elif G.has_node(row['Cited names'][node]):
#                    G.add_edge(row['Sender'], row['Cited names'][node], weight = weight_citation_made, color = 'b')
#                else:
#                    G.add_node(row['Cited names'][node])
#                    G.add_edge(row['Sender'], row['Cited names'][node], weight = weight_citation_made, color='b')


            
        for node in range(0, len(row['Cited names'])):
            if G.has_edge(row['Sender'], row['Cited names'][node]):
                G[row['Sender']][row['Cited names'][node]]['weight'] += weight_citation_made
            else:
                if G.has_node(row['Cited names'][node]):
                    G.add_edge(row['Sender'], row['Cited names'][node], weight = weight_citation_made, color = 'b')
                else:
                    G.add_node(row['Cited names'][node])
                    G.add_edge(row['Sender'], row['Cited names'][node], weight = weight_citation_made, color='b')
            
            if G.has_edge(row['Receiver'], row['Cited names'][node]):
                G[row['Receiver']][row['Cited names'][node]]['weight'] += weight_citation_received
            else:
                if G.has_node(row['Cited names'][node]):
                    G.add_edge(row['Receiver'], row['Cited names'][node], weight = weight_citation_received, color = 'b')
                else:
                    G.add_node(row['Cited names'][node])
                    G.add_edge(row['Receiver'], row['Cited names'][node], weight = weight_citation_received, color='b')

             


edges_corres = [x for x in G.edges if (x[0] in correspondents and x[1] in correspondents)]
edges_top_corres = [x for x in G.edges if (x[0] in top_correspondents and x[1] in top_correspondents)]

pos=nx.spring_layout(G, k=.05,iterations=100)#, seed = 7)

figure(figsize=(100,100))#, dpi=400)
weights = [.1*G[u][v]['weight'] for u,v in G.edges]

nx.draw_networkx_nodes(G, pos, node_size=170)#, nodelist=correspondents)
nx.draw_networkx_labels(G, pos, font_size=17, font_family="sans-serif", labels=labels_only_cited)
nx.draw_networkx_labels(G, pos, font_size=17, font_family="sans-serif", font_color='tab:blue', labels=labels_wo_top)
nx.draw_networkx_edges(G, pos, width=weights, alpha=.5, edge_color='tab:blue')#,edgelist=edges_corres)

nx.draw_networkx_labels(G, pos, labels=labels_top_corres, font_size=30, font_family="serif", font_weight='bold', font_color='tab:red')


plt.show()
#plt.savefig("Graph.svg")

# Below is a sandbox

In [ ]:
G.remove_node('Viviani Vincenzo') 
color_edge = [G[u][v]['color'] for u,v in G.edges]
color_map = [G.nodes[u]['color'] for u in G.nodes] 
figure(figsize=(100,60))
nx.draw_networkx(G, with_labels=True, node_color = color_map,edge_color = color_edge)

In [ ]:
#net = Network(notebook=True,height='750px', width='100%', bgcolor='#00000', font_color='black')
#net.from_nx(G)
#net.show_buttons(filter_=['physics'])
#net.show("test.html")

In [ ]:
##Strongly connected components:
    #There is a directed path between node A to node B and another from node B to node A.
#Weakly connected components:
    #There is a directed path from node A to node B but not necessarily from node B to node A.

def get_strongly_cc(G, node):
    """ get storngly connected component of node""" 
    for cc in nx.strongly_connected_components(G):
        if node in cc:
            return cc
    else:
        return set()

def get_weakly_cc(G, node):
    """ get weakly connected component of node""" 
    for cc in nx.weakly_connected_components(G):
        if node in cc:
            return cc
    else:
        return set()


weak_component = get_weakly_cc(G, node)  # Weakly connected component of node in G
strong_component = get_strongly_cc(G, node)  # Strongly connected component of node in G

In [ ]:
get_weakly_cc(G, 'Southwell Robert')

In [ ]:
#nx.node_connected_component(G, 'Southwell Robert')

In [ ]:
nx.transitivity(G)

In [ ]:

sub_graphs_test = nx.connected_components(G)
while True:
    try:
        print(next(sub_graphs_test))
    except StopIteration:
        break

In [ ]:
def most_central_edge(G):
    centrality = nx.edge_betweenness_centrality(G,weight = 'weight')
    max_cent = max(centrality.values())
    return max(centrality, key=centrality.get)

In [ ]:
from networkx.algorithms import community
communities = community.girvan_newman(G, most_valuable_edge=most_central_edge)
node_groups = []
for com in next(communities):
    node_groups.append(list(com))

color_map = []
for node in G:
    for i in range(len(node_groups)):
        if node in node_groups[i]:
            color_map.append(255*(i+1)/(len(node_groups)+1))
                             
figure(figsize=(50,30))
nx.draw(G, node_color=color_map, with_labels=True)
plt.show()

In [ ]:
def _position_communities(g, partition, **kwargs):

    # create a weighted graph, in which each node corresponds to a community,
    # and each edge weight to the number of edges between communities
    between_community_edges = _find_between_community_edges(g, partition)

    communities = set(partition.values())
    hypergraph = nx.DiGraph()
    hypergraph.add_nodes_from(communities)
    for (ci, cj), edges in between_community_edges.items():
        hypergraph.add_edge(ci, cj, weight=len(edges))

    # find layout for communities
    pos_communities = nx.spring_layout(hypergraph, **kwargs)

    # set node positions to position of community
    pos = dict()
    for node, community in partition.items():
        pos[node] = pos_communities[community]

    return pos

def _find_between_community_edges(g, partition):

    edges = dict()

    for (ni, nj) in g.edges():
        ci = partition[ni]
        cj = partition[nj]

        if ci != cj:
            try:
                edges[(ci, cj)] += [(ni, nj)]
            except KeyError:
                edges[(ci, cj)] = [(ni, nj)]

    return edges

def _position_nodes(g, partition, **kwargs):
    """
    Positions nodes within communities.
    """

    communities = dict()
    for node, community in partition.items():
        try:
            communities[community] += [node]
        except KeyError:
            communities[community] = [node]

    pos = dict()
    for ci, nodes in communities.items():
        subgraph = g.subgraph(nodes)
        pos_subgraph = nx.spring_layout(subgraph, **kwargs)
        pos.update(pos_subgraph)

    return pos




In [ ]:
#TODO:  create graph for each community, add edges with edge x factor of 10 between nodes of same community
def community_layout(g, partition):
    """
    Compute the layout for a modular graph.


    Arguments:
    ----------
    g -- networkx.Graph or networkx.DiGraph instance
        graph to plot

    partition -- dict mapping int node -> int community
        graph partitions


    Returns:
    --------
    pos -- dict mapping int node -> (float x, float y)
        node positions

    """

    pos_communities = _position_communities(g, partition, scale=15)

    pos_nodes = _position_nodes(g, partition, scale=1)

    # combine positions
    pos = dict()
    for node in g.nodes():
        pos[node] = pos_communities[node] + pos_nodes[node]

    return pos

# Community detection

In [ ]:
from community import community_louvain


In [ ]:
from community import community_louvain

#code to draw graph of partitions
partition = community_louvain.best_partition(G)
pos = community_layout(G, partition)
node_colors = []
for node in G:
    for i in range(max(partition.values())+1):
        if partition[node] == i:
            node_colors.append(255*(i+1)/(max(partition.values())+1))



color_map = [G.nodes[u]['color'] for u in G.nodes]
figure(figsize=(50,30))
nx.draw(G, pos, node_color=node_colors, edge_color = color_map, with_labels = True)
plt.show()

In [ ]:
G.remove_node('Viviani Vincenzo')
G.remove_nodes_from(list(nx.isolates(G)))
#code to draw graph of partitions
partition = community_louvain.best_partition(G)
pos = community_layout(G, partition)
node_colors = []
for node in G:
    for i in range(max(partition.values())+1):
        if partition[node] == i:
            node_colors.append(255*(i+1)/(max(partition.values())+1))



color_map = [G.nodes[u]['color'] for u in G.nodes]
figure(figsize=(50,30))
nx.draw(G, pos, node_color=node_colors, edge_color = color_map, with_labels = True)
plt.show()

In [ ]:
G.remove_node('Southwell Robert')
G.remove_nodes_from(list(nx.isolates(G)))
#code to draw graph of partitions
partition = community_louvain.best_partition(G)
pos = community_layout(G, partition)
node_colors = []
for node in G:
    for i in range(max(partition.values())+1):
        if partition[node] == i:
            node_colors.append(255*(i+1)/(max(partition.values())+1))



color_map = [G.nodes[u]['color'] for u in G.nodes]
figure(figsize=(50,30))
nx.draw(G, pos, node_color=node_colors, edge_color = color_map, with_labels = True)
plt.show()

In [ ]:
#TEST YEAR 1660
year_1660 = filtred.loc[filtred['Year'] == '1660']
#creating a list of  the starting nodes for the network
starting_nodes = year_1660['Sender'].unique()
starting_nodes

#creating the graph and adding the starting nodes
G = nx.Graph()
for node in starting_nodes:
    G.add_node(node, color = 'red')


#list of the main persons Viviani exchanges with    
correspondents = filtred['Sender'].append(filtred['Receiver']).unique()      
    
    
#going through each row and adding a node/edge from names cited (destinataire is always in the names cited)
for index, row in year_1660.iterrows():
    for node in range(len(row['Cited names'])):
        if row['Sender'] == row['Cited names'][node]:
            pass
        else:
            if G.has_edge(row['Sender'], row['Cited names'][node]):
                G[row['Sender']][row['Cited names'][node]]['weight'] += 1
            else:
                if row['Cited names'][node] in correspondents:
                    G.add_node(row['Cited names'][node], color = 'green')
                    if G.nodes[row['Sender']]['color'] == 'green':
                        G.add_edge(row['Sender'], row['Cited names'][node], weight = 5, color = 'g')
                    else:
                        G.add_edge(row['Sender'], row['Cited names'][node], weight = 5, color = 'r')
                else:
                    G.add_node(row['Cited names'][node], color = 'blue')
                    G.add_edge(row['Sender'], row['Cited names'][node], weight = 1, color = 'b')

#setting the color map for the nodes                   
color_edge = [G[u][v]['color'] for u,v in G.edges]
color_map = [G.nodes[u]['color'] for u in G.nodes]

figure(figsize=(50,30))
nx.draw_networkx(G, with_labels=True, node_color = color_map ,edge_color= color_edge)

In [ ]:
#code to draw graph of partitions
partition = community_louvain.best_partition(G)
pos = community_layout(G, partition)
node_colors = []
for node in G:
    for i in range(max(partition.values())+1):
        if partition[node] == i:
            node_colors.append(255*(i+1)/(max(partition.values())+1))



color_map = [G.nodes[u]['color'] for u in G.nodes]
figure(figsize=(100,60))
nx.draw(G, pos, node_color=node_colors, edge_color = color_map, with_labels = True)
plt.show()

In [ ]:
G.remove_node('Viviani Vincenzo')
G.remove_nodes_from(list(nx.isolates(G)))
#code to draw graph of partitions
partition = community_louvain.best_partition(G)
pos = community_layout(G, partition)
node_colors = []
for node in G:
    for i in range(max(partition.values())+1):
        if partition[node] == i:
            node_colors.append(255*(i+1)/(max(partition.values())+1))



color_map = [G.nodes[u]['color'] for u in G.nodes]
figure(figsize=(100,60))
nx.draw(G, pos, node_color=node_colors, edge_color = color_map, with_labels = True)
plt.show()

In [ ]:
G.remove_node('Southwell Robert')
G.remove_nodes_from(list(nx.isolates(G)))
#code to draw graph of partitions
partition = community_louvain.best_partition(G)
pos = community_layout(G, partition)
node_colors = []
for node in G:
    for i in range(max(partition.values())+1):
        if partition[node] == i:
            node_colors.append(255*(i+1)/(max(partition.values())+1))



color_map = [G.nodes[u]['color'] for u in G.nodes]
figure(figsize=(100,60))
nx.draw(G, pos, node_color=node_colors, edge_color = color_map, with_labels = True)
plt.show()

In [ ]:
#TEST YEAR 1695
year_1695 = filtred.loc[filtred['Year'] == '1695']
#creating a list of  the starting nodes for the network
starting_nodes = year_1695['Sender'].unique()
starting_nodes

#creating the graph and adding the starting nodes
G = nx.Graph()
for node in starting_nodes:
    G.add_node(node, color = 'red')


#list of the main persons Viviani exchanges with    
correspondents = filtred['Sender'].append(filtred['Receiver']).unique()      
    
    
#going through each row and adding a node/edge from names cited (destinataire is always in the names cited)
for index, row in year_1695.iterrows():
    for node in range(len(row['Cited names'])):
        if row['Sender'] == row['Cited names'][node]:
            pass
        else:
            if G.has_edge(row['Sender'], row['Cited names'][node]):
                G[row['Sender']][row['Cited names'][node]]['weight'] += 1
            else:
                if row['Cited names'][node] in correspondents:
                    G.add_node(row['Cited names'][node], color = 'green')
                    if G.nodes[row['Sender']]['color'] == 'green':
                        G.add_edge(row['Sender'], row['Cited names'][node], weight = 5, color = 'g')
                    else:
                        G.add_edge(row['Sender'], row['Cited names'][node], weight = 5, color = 'r')
                else:
                    G.add_node(row['Cited names'][node], color = 'blue')
                    G.add_edge(row['Sender'], row['Cited names'][node], weight = 1, color = 'b')

#setting the color map for the nodes                   
color_edge = [G[u][v]['color'] for u,v in G.edges]
color_map = [G.nodes[u]['color'] for u in G.nodes]

figure(figsize=(50,30))
nx.draw_networkx(G, with_labels=True, node_color = color_map ,edge_color= color_edge)

In [ ]:
G.remove_node('Viviani Vincenzo')
G.remove_nodes_from(list(nx.isolates(G)))
#code to draw graph of partitions
partition = community_louvain.best_partition(G)
pos = community_layout(G, partition)
node_colors = []
for node in G:
    for i in range(max(partition.values())+1):
        if partition[node] == i:
            node_colors.append(255*(i+1)/(max(partition.values())+1))



color_map = [G.nodes[u]['color'] for u in G.nodes]
figure(figsize=(50,30))
nx.draw(G, pos, node_color=node_colors, edge_color = color_map, with_labels = True)
plt.show()

In [ ]:
#TEST YEAR 1662
year_1662 = filtred.loc[filtred['Year'] == '1662']
#creating a list of  the starting nodes for the network
starting_nodes = year_1662['Sender'].unique()
starting_nodes

#creating the graph and adding the starting nodes
G = nx.Graph()
for node in starting_nodes:
    G.add_node(node, color = 'red')


#list of the main persons Viviani exchanges with    
correspondents = filtred['Sender'].append(filtred['Receiver']).unique()      
    
    
#going through each row and adding a node/edge from names cited (destinataire is always in the names cited)
for index, row in year_1662.iterrows():
    for node in range(len(row['Cited names'])):
        if row['Sender'] == row['Cited names'][node]:
            pass
        else:
            if G.has_edge(row['Sender'], row['Cited names'][node]):
                G[row['Sender']][row['Cited names'][node]]['weight'] += 1
            else:
                if row['Cited names'][node] in correspondents:
                    G.add_node(row['Cited names'][node], color = 'green')
                    if G.nodes[row['Sender']]['color'] == 'green':
                        G.add_edge(row['Sender'], row['Cited names'][node], weight = 1, color = 'g')
                    else:
                        G.add_edge(row['Sender'], row['Cited names'][node], weight = 1, color = 'r')
                else:
                    G.add_node(row['Cited names'][node], color = 'blue')
                    G.add_edge(row['Sender'], row['Cited names'][node], weight = 1, color = 'b')

#setting the color map for the nodes                   
color_edge = [G[u][v]['color'] for u,v in G.edges]
color_map = [G.nodes[u]['color'] for u in G.nodes]

figure(figsize=(50,30))
nx.draw_networkx(G, with_labels=True, node_color = color_map ,edge_color= color_edge)

In [ ]:
G.remove_node('Viviani Vincenzo')
G.remove_nodes_from(list(nx.isolates(G)))
#code to draw graph of partitions
partition = community_louvain.best_partition(G)
pos = community_layout(G, partition)
node_colors = []
for node in G:
    for i in range(max(partition.values())+1):
        if partition[node] == i:
            node_colors.append(255*(i+1)/(max(partition.values())+1))



color_map = [G.nodes[u]['color'] for u in G.nodes]
figure(figsize=(50,30))
nx.draw(G, pos, node_color=node_colors, edge_color = color_map, with_labels = True)
plt.show()